In [1]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from collections import Counter
from PyEMD import EEMD, EMD, Visualisation
import math
import pandas as pd

In [2]:
# 定义数据分布展示

def show_plot(data,show_num):
    data_list = []
    for a in data[:show_num]:
        data_list.append(a)#append函数会在数组后加上相应的元素
    return data_list

In [3]:
# 计算无量纲特征量

pstf_list=[]
def  psfeatureTime(data):
    #有量纲
    #均值
    df_mean = sum(data) / len(data)
    #绝对均值
    df_absmean = sum(abs(data)) / len(data)
    #方根幅值
    df_r = pow(np.mean(np.sqrt(abs(data))),2)
    #均方根值
    df_rms = np.sqrt(np.mean(pow(data,2)))
    #峰值
    df_max = max(abs(data))
    #偏度值
    df_alpha = np.mean(pow(data,3))
    #峭度值
    df_beta = np.mean(pow(data,4))

    #无量纲
    #峰值因子
    df_cf = df_max/df_rms
    #脉冲因子
    df_if = df_max/df_absmean
    #裕度因子
    df_clf = df_max/df_r
    #波形因子
    df_sf = df_rms/df_absmean
    #偏度因子
    df_k3 = df_alpha/pow(df_rms,3)
    #峭度因子
    df_k4 = df_beta/pow(df_rms,4)
    
    return df_cf,df_if,df_clf,df_sf,df_k3,df_k4

In [4]:
# 定义样本分割函数
def data_load(path,cut_num,shaft,label):
    """
    path：数据地址
    cut_num：每份样本数量
    shaft：轴，0为输入轴，3为中间轴
    label:数据标签
    """
    
    data = loadmat(path)
    # 返回list
    list_r = []
    # 原始数据
    data_X = data['data'][:,shaft]
    data_X = data_X.reshape(len(data_X),1)
    data_Y = data['data'][:,shaft+1]
    data_Y = data_Y.reshape(len(data_Y),1)
    data_Z = data['data'][:,shaft+2]
    data_Z = data_Z.reshape(len(data_Z),1)
    data_XYZ = np.sqrt(pow(data_X,2) + pow(data_Y,2) + pow(data_Z,2))

    # 数据归一化
    # 归一化X
    scaler = MinMaxScaler()
    list_X_n = scaler.fit_transform(data_X)
    # 归一化Y
    scaler = MinMaxScaler()
    list_Y_n = scaler.fit_transform(data_Y)
    # 归一化Z
    scaler = MinMaxScaler()
    list_Z_n = scaler.fit_transform(data_Z)
    
    # 构建一维数组
    list_X = []
    for x in list_X_n:
        list_X.append(x[0])
        
    list_Y = []
    for y in list_Y_n:
        list_Y.append(y[0])

    list_Z = []
    for z in list_Z_n:
        list_Z.append(z[0])
        
    # 将x，y, z数据放入list
    for x,y,z in zip(list_X,list_Y,list_Z):
        list_r.append([x,y,z])
    
    data_cut = []
    label_cut = []
    characteristic_X = []
    characteristic_Y = []
    characteristic_Z = []
    data_ret = []
    
    # 分割数据
    for i in range(0,int(len(list_X_n)/cut_num)):
        data_cut.append(list_r[i * cut_num : (i+1) * cut_num])
        label_cut.append(label)
#     return data_cut,label_cut

    # emd
    emd = EMD()
    for i in range(0,len(data_cut)):
        list_tmp = []
        emd.emd(np.asarray(data_cut,dtype = 'float')[i,:,0])
        imfs, res = emd.get_imfs_and_residue()
        for j in range(0,6):
            values = psfeatureTime(imfs[j])
            for k in range(0,6):
                list_tmp.append(values[k])
        characteristic_X.append(list_tmp)
        
    for i in range(0,len(data_cut)):
        list_tmp = []
        emd.emd(np.asarray(data_cut,dtype = 'float')[i,:,1])
        imfs, res = emd.get_imfs_and_residue()
        for j in range(0,6):
            values = psfeatureTime(imfs[j])
            for k in range(0,6):
                list_tmp.append(values[k])
        characteristic_Y.append(list_tmp)
        
    for i in range(0,len(data_cut)):
        list_tmp = []
        emd.emd(np.asarray(data_cut,dtype = 'float')[i,:,2])
        imfs, res = emd.get_imfs_and_residue()
        for j in range(0,6):
            values = psfeatureTime(imfs[j])
            for k in range(0,6):
                list_tmp.append(values[k])
        characteristic_Z.append(list_tmp)
        
    for x,y,z in zip(characteristic_X,characteristic_Y,characteristic_Z):
        data_ret.append([x,y,z])
    
    return data_ret,label_cut

# label0: 正常轴承

In [5]:
data_0,label_0 = data_load('data/5-normal1-2000rpm-50Nm.mat',2000,0,0)

In [6]:
print("处理后内圈样本shape：",np.asarray(data_0,dtype = 'float').shape)
print("label数：",len(label_0))

处理后内圈样本shape： (480, 3, 36)
label数： 480


# label1: 输入轴减速器端轴承内滚道故障 2000rpm

In [7]:
data_1,label_1 = data_load('data/1-bearingfault_JSQ1-2000rpm-50Nm.mat',2000,3,1)

In [8]:
print("处理后内圈样本shape：",np.asarray(data_1,dtype = 'float').shape)
print("label数：",len(label_1))

处理后内圈样本shape： (480, 3, 36)
label数： 480


# label2: 输入轴电机端轴承内滚道故障 2000rpm

In [9]:
data_2,label_2 = data_load('data/2-bearingfault_DJ1-2000rpm-50Nm.mat',2000,6,2)

In [10]:
print("处理后内圈样本shape：",np.asarray(data_2,dtype = 'float').shape)
print("label数：",len(label_2))

处理后内圈样本shape： (480, 3, 36)
label数： 480


# label3: 中间轴减速器端轴承滚子故障 2000rpm

In [11]:
data_3,label_3 = data_load('data/3-bearingfault_JSQ1-2000rpm-50Nm.mat',2000,0,3)

In [12]:
print("处理后内圈样本shape：",np.asarray(data_3,dtype = 'float').shape)
print("label数：",len(label_3))

处理后内圈样本shape： (480, 3, 36)
label数： 480


# label4: 输入轴减速器端轴承保持架故障 2000rpm

In [13]:
data_4,label_4 = data_load('data/4-GF-BF1-2000rpm-50Nm.mat',2000,3,4)

In [14]:
print("处理后内圈样本shape：",np.asarray(data_4,dtype = 'float').shape)
print("label数：",len(label_4))

处理后内圈样本shape： (480, 3, 36)
label数： 480


In [22]:
data_train = np.asarray(data_0[:] + data_1[:] + data_2[:] + data_3[:] + data_4[:],dtype = 'float64')
data_train = data_train.transpose(0,2,1)
label = np.asarray(label_0[:] + label_1[:] + label_2[:] + label_3[:] + label_4[:],dtype = 'int64')

print("处理后样本shape：",data_train.shape)
print("处理后数据类别分布：",Counter(label))

# 保存数据
np.save("train_data/train_data_2000_emd.npy",data_train)
np.save("train_data/label_2000_emd.npy",label)
print("数据保存成功，位置：/train_data/")

处理后样本shape： (2400, 36, 3)
处理后数据类别分布： Counter({0: 480, 1: 480, 2: 480, 3: 480, 4: 480})
数据保存成功，位置：/train_data/


In [2]:
import numpy as np

In [6]:
data_train = np.load('train_data/label_2000_emd.npy')

In [8]:

data_train.shape

(2400,)